# 텍스트 데이터 전처리 하기

딥러닝 모형에서는 텍스트를 수치로 변환하여 처리해야 한다.



# BoW(Bag of Word)

텍스트나 단어를 사용하기 전에 수치 형태로 변환하는 전처리 과정을 거쳐야 한다. 텍스트를 수치 벡터로 표현하는 방법으로 BoW가 있다.

1. 전체 문서를 고유한 token으로 변환한다.
2. 특정 문서에서 각 단어가 얼마나 자주 등장하는지 세어서 문서의 특성 벡터를 생성한다. 

In [0]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

### CountVectorizer

CountVectorizer는 이러한 작업을 하기 위한 다음과 같은 인수를 가질 수 있다.

* stop_words : 문자열 {‘english’}, 리스트 또는 None (디폴트)
* stop words 목록.‘english’이면 영어용 스탑 워드 사용.
* analyzer : 문자열 {‘word’, ‘char’, ‘char_wb’} 또는 함수
단어 n-그램, 문자 n-그램, 단어 내의 문자 n-그램
* token_pattern : string
토큰 정의용 정규 표현식
* tokenizer : 함수 또는 None (디폴트)
토큰 생성 함수 .
* ngram_range : (min_n, max_n) 튜플
n-그램 범위
* max_df : 정수 또는 [0.0, 1.0] 사이의 실수. 디폴트 1
단어장에 포함되기 위한 최대 빈도
* min_df : 정수 또는 [0.0, 1.0] 사이의 실수. 디폴트 1
단어장에 포함되기 위한 최소 빈도

어휘 사전의 내용을 출력해 보면 다음과 같다.

In [2]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [3]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


# Load Text

RNN에서 텍스트 데이터 전처리는 BoW와는 달리 고유한 단어의 집합만 관심 대상이고 부수적으로 생성된 단어의 빈도는 필요하지 않다.

tf.data.TextLineDataset를 사용하면 텍스트 파일로 부터 데이터 셋을 생성할 수 있다. 

서로 다른 작가가 호머의 일리아드를 영어로 번역한 3개의 텍스트를 읽어 들여 적절하게 데이터 전처리를 수행하고 데이터 셋으로 생성한다.

In [4]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

import tensorflow_datasets as tfds
import os

TensorFlow 2.x selected.
2.0.0


In [5]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
  text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)
  
parent_dir = os.path.dirname(text_dir)

parent_dir

'/root/.keras/datasets'

### Load text into datasets

tf.data.Dataset.map을 사용하여 labeler function을 적용하면 데이터 셋의 각 데이터에 대해서 (example, label) 쌍으로 리턴해 준다.


In [0]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)  

labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
  labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  labeled_data_sets.append(labeled_dataset)

각 각의 labeled 데이터 셋을 하나의 데이터 셋으로 합친다.

In [0]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [0]:
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)
  
all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

tf.data.Dataset.take 를 사용하여 데이터 셋에서 데이터를 읽어 들여 화면에 출력한다.

In [9]:
for ex in all_labeled_data.take(5):
  print(ex)

(<tf.Tensor: id=74, shape=(), dtype=string, numpy=b'and killed Iphidamas by striking him on the neck. So there the poor'>, <tf.Tensor: id=75, shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: id=76, shape=(), dtype=string, numpy=b'woe betide them, pressing you hard about the city that you have thought'>, <tf.Tensor: id=77, shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: id=78, shape=(), dtype=string, numpy=b'By word and hand, to bear me harmless through.'>, <tf.Tensor: id=79, shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: id=80, shape=(), dtype=string, numpy=b'battle--for his age stayed him not. He raised himself on his elbow and'>, <tf.Tensor: id=81, shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: id=82, shape=(), dtype=string, numpy=b'Echius, in the front of the battle, while Agenor slew Clonius. Paris'>, <tf.Tensor: id=83, shape=(), dtype=int64, numpy=2>)


# Build vocabulary

tfds.features.text.Tokenizer를 사용하여 각 데이터를 토큰화 한다.

In [10]:
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

17178

### Encode examples
encode method 는 문자열을 입력으로 받아 integer 리스트를 리턴해 준다.

In [0]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [12]:
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)

b'and killed Iphidamas by striking him on the neck. So there the poor'


In [13]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

[9922, 7231, 10378, 11522, 12215, 12483, 14814, 695, 10032, 11669, 2677, 695, 3664]


In [0]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label


# tf.py_function을 사용하여 tensorflow에서 파이썬 함수를 호출한다.
def encode_map_fn(text, label):
  return tf.py_function(encode, inp=[text, label], Tout=(tf.int64, tf.int64))

all_encoded_data = all_labeled_data.map(encode_map_fn)

In [15]:
list(all_encoded_data.take(1))

[(<tf.Tensor: id=99331, shape=(13,), dtype=int64, numpy=
  array([ 9922,  7231, 10378, 11522, 12215, 12483, 14814,   695, 10032,
         11669,  2677,   695,  3664])>,
  <tf.Tensor: id=99332, shape=(), dtype=int64, numpy=2>)]

 각 텍스트 줄의 단어 수는 다르다. 따라서 tf.data.Dataset.padded_batch을 사용 하여 예제를 동일한 크기로 채운다.

In [0]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes=([-1],[]))

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE, padded_shapes=([-1],[]))

In [17]:
sample_text, sample_labels = next(iter(train_data))

print(sample_text.shape, sample_labels.shape)

sample_text[0], sample_labels[0]


(64, 15) (64,)


(<tf.Tensor: id=248191, shape=(15,), dtype=int64, numpy=
 array([11237,  5190,  1075,  2118,  9523, 10869,  5562,     0,     0,
            0,     0,     0,     0,     0,     0])>,
 <tf.Tensor: id=248195, shape=(), dtype=int64, numpy=0>)